In [2]:
import os
import time
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

In [ ]:
NUMBER_OF_PAGES = 2030

In [4]:
saving_path = "../parsing_results/output_links_270525.csv"

In [5]:

def get_link_from_page(url: str, csv_file=saving_path):

    # Send HTTP GET request
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page: {url}")

    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the section and extract all <a> tags inside it
    section = soup.find('section', class_='a-list a-search-list a-list-with-favs')
    links = []

    if section:
        for card in section.find_all('div', class_='a-card a-storage-live ddl_product ddl_product_link not-colored is-visible'):
            link_tag = card.find('a', class_='a-card__image')
            if link_tag and link_tag.get('href'):
                full_url = link_tag['href']
                if full_url.startswith('/'):
                    full_url = f"https://krisha.kz/{full_url.lstrip('/')}"
                links.append(full_url)

    # Save to CSV

    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        if file.tell() == 0:
            writer.writerow(["Link"])
        
        for link in links:
            writer.writerow([link])

In [6]:
get_link_from_page("https://krisha.kz/prodazha/uchastkov/", "test.csv")

In [10]:


page_numbers = range(1, NUMBER_OF_PAGES)  # Pages 1 to last

for page in tqdm(page_numbers):
    # if page % 50 == 0:
    #     print(f'page {page} as successfully parsed')
    url = f"https://krisha.kz/prodazha/uchastkov/?page={page}"
    try:
        get_link_from_page(url, saving_path)
        time.sleep(1)  # Pause for 1 seconds between requests
    except Exception as e:
        print(f"Error on {url}: {e}")
        time.sleep(5)  # Longer pause if error occurs

 12%|█▏        | 242/2029 [05:46<45:19,  1.52s/it]

Error on https://krisha.kz/prodazha/uchastkov/?page=243: Response ended prematurely


100%|██████████| 2029/2029 [55:49<00:00,  1.65s/it] 
